# Augmentation Method Metrics on Hold-out Sets

In [1]:
import torch
import torch.nn as nn
from torch.cuda import amp
from torch.utils.data.dataloader import Dataset, DataLoader

from torchvision import transforms

from models import Net

from eval_metrics import *
from models import *
from operations import *
from datasets import MultilabelDataset
import os

from glob import glob

import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc, precision_recall_curve

from tqdm import tqdm_notebook

image_classes = ['cored','diffuse','CAA']

In [2]:
@torch.no_grad()
def validate(model, dataloader, dataset_size,device= torch.device('cpu')):
    # set model to eval mode
    model.eval()
    running_loss = 0.0
    running_corrects = torch.zeros(model.num_classes).to(device)
    running_preds = torch.Tensor(0).to(device)
    running_predictions = torch.Tensor(0).to(device)
    running_labels = torch.Tensor(0)
    
    for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):
        # get inputs and move to specified device
        inputs, labels, raw_labels, names = data
        running_labels = torch.cat([running_labels, labels])
        inputs, labels = inputs.to(device), labels.to(device)

 
        with amp.autocast():
            outputs = model(inputs)

        preds = torch.sigmoid(outputs)
        predictions = (preds>0.5).long()
        running_predictions = torch.cat([running_predictions, predictions])
        running_preds = torch.cat([running_preds, preds])
        running_corrects += torch.sum(predictions==labels, 0)

    running_preds = running_preds.to('cpu')
    running_predictions = running_predictions.to('cpu')
    running_corrects = running_corrects.to('cpu')
    epoch_acc = running_corrects / dataset_size

#     # calculate precision and recall
#     _, pr_auc_macro_list = macro_avg_auc(running_preds, running_labels, type='prc')
#     _, roc_auc_macro_list = macro_avg_auc(running_preds, running_labels, type='roc')
    
    return epoch_acc, running_preds, running_predictions, running_labels


In [3]:
def plot_roc(preds, label, image_classes, phase, size=20, path=None):
    colors = ['pink','c','deeppink', 'b', 'g', 'm', 'y', 'r', 'k']
    fig = plt.figure(figsize=(1.2*size, size))
    ax = plt.axes()
    for i in range(preds.shape[1]):
        fpr, tpr, _ = roc_curve(label[:,i].ravel(), preds[:,i].ravel())
        lw = 0.2*size
        # Plot all ROC curves
        ax.plot([0, 1], [0, 1], 'k--', lw=lw, label='random')
        ax.plot(fpr, tpr,
                 label='ROC-curve of {}'.format(image_classes[i])+ '( area = {0:0.3f})'
                ''.format(auc(fpr, tpr)),
                  color=colors[(i+preds.shape[1])%len(colors)], linewidth=lw)
       
    
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=1.8*size)
    ax.set_ylabel('True Positive Rate', fontsize=1.8*size)
    ax.set_title('Receiver operating characteristic Curve ({})'.format(phase), fontsize=1.8*size, y=1.01)
    ax.legend(loc=0, fontsize=1.5*size)
    ax.xaxis.set_tick_params(labelsize=1.6*size, size=size/2, width=0.2*size)
    ax.yaxis.set_tick_params(labelsize=1.6*size, size=size/2, width=0.2*size)
    
    if path != None:
        if os.path.isfile(path):
            print('{} already exists, not saving image'.format(path))
        else:
            fig.savefig(path)
            plt.close(fig)
            print('saved')

    
def plot_prc(preds, label, image_classes, phase, size=20, path=None):
    colors = ['pink','c','deeppink', 'b', 'g', 'm', 'y', 'r', 'k']
    
    fig = plt.figure(figsize=(1.2*size,size))
    ax = plt.axes()
    
    for i in range(preds.shape[1]):
        rp = (label[:,i]>0).sum()/len(label)
        precision, recall, _ = precision_recall_curve(label[:,i].ravel(), preds[:,i].ravel())
        
        lw=0.2*size
    
        ax.plot(recall, precision,
                 label='PR-curve of {}'.format(image_classes[i])+ '( area = {0:0.3f})'
                ''.format(auc(recall, precision)),
                 color=colors[(i+preds.shape[1])%len(colors)], linewidth=lw)

        ax.plot([0, 1], [rp, rp], 'k--', color=colors[(i+preds.shape[1])%len(colors)], lw=lw, label='random')
   
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Recall', fontsize=1.8*size)
    ax.set_ylabel('Precision', fontsize=1.8*size)
    ax.set_title('Precision-Recall curve ({})'.format(phase), fontsize=1.8*size, y=1.01)
    ax.legend(loc="lower left", bbox_to_anchor=(0.01, 0.1), fontsize=1.5*size)
    ax.xaxis.set_tick_params(labelsize=1.6*size, size=size/2, width=0.2*size)
    ax.yaxis.set_tick_params(labelsize=1.6*size, size=size/2, width=0.2*size)
    
    if path != None:
        if os.path.isfile(path):
            print('{} already exists, not saving image'.format(path))
        else:
            fig.savefig(path)
            plt.close(fig)
            print('saved')

In [5]:
@torch.no_grad()
def run_and_plot_metrics(base_dir, test_set = 'combined', best_run = 'loss',
                         model_names=None, runs=[0,1,2]):
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    if model_names is None:
        # running all models. Get all subdirectories in base directory
        model_names = os.listdir(base_dir)
    else:
        model_names = list(model_names)
    
    # load in dataset
    norm = np.load('./utils/normalization.npy', allow_pickle = True).item()
    data_trans = get_transformations(norm, 'none')
    data_trans = data_trans['test']
    data_dirs = get_data_dirs()
    data_dir = data_dirs['test']
    
    csv_paths = get_csv_paths('none')
    if test_set == 'orig':
        csv_path = csv_paths['test']
    elif test_set == 'alt':
        csv_path = csv_paths['test_alt']
    elif test_set == 'combined':
        csv_path = csv_paths['test_combined']
    elif test_set == 'dev':
        csv_path = csv_paths['dev']
        data_dir = data_dirs['dev']
    
    dataset = MultilabelDataset(csv_path, data_dir, data_trans)
    dataloader = DataLoader(dataset, batch_size=64, num_workers=0, shuffle=False)
    for model_name in tqdm_notebook(model_names, total=len(model_names)):
        save_dir = os.path.join(base_dir, model_name, 'results','auc_curves')
        os.makedirs(save_dir, exist_ok=True)
        
        for run in runs:
            # initialise model
            ckpt_path = os.path.join(base_dir, model_name, 'models','best',f'{run}_{best_run}.pt')
            if os.path.exists(ckpt_path):
                ckpt = torch.load(ckpt_path)
                model = Net()
                model.load_state_dict(ckpt['model_state_dict'])
                model.to(device)
                model.eval()

                acc, pred, prediction, target = validate(model, dataloader, len(dataset), device=device)

                label = target.numpy()
                preds = pred.numpy()

                output = {}
                for i in range(3):
                    fpr, tpr, _ = roc_curve(label[:,i].ravel(), preds[:,i].ravel())

                    precision, recall, _ = precision_recall_curve(label[:,i].ravel(), preds[:,i].ravel())

                    output['{} fpr'.format(image_classes[i])] = fpr
                    output['{} tpr'.format(image_classes[i])] = tpr
                    output['{} precision'.format(image_classes[i])] = precision
                    output['{} recall'.format(image_classes[i])] = recall

                plot_roc(pred.numpy(), target.numpy(), image_classes, 'test', size=30,
                         path=os.path.join(save_dir, f'roc_{test_set}_{best_run}_{run}.png'))
                plot_prc(pred.numpy(), target.numpy(), image_classes, 'test', size=30,
                         path=os.path.join(save_dir, f'prc_{test_set}_{best_run}_{run}.png'))

In [8]:
# running on dev set for GAN-related
base_dir = 'D:/ucl_masters_data/project/augmentation_experiments'
model_names = ['U_gan_A_none_L_simple', 'U_gan_0.5_A_none_L_simple']
run_and_plot_metrics(base_dir, test_set='dev',best_run='loss', model_names=model_names)

<ipython-input-5-1a9e5bedf772>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model_name in tqdm_notebook(model_names, total=len(model_names)):


  0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/135 [00:00<?, ?it/s]

saved
saved


...

  0%|          | 0/135 [00:00<?, ?it/s]

saved
saved


<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/135 [00:00<?, ?it/s]

saved
saved


In [7]:
base_dir = 'D:/ucl_masters_data/project/augmentation_experiments'
run_and_plot_metrics(base_dir, test_set='orig',best_run='loss', model_names=None)

<ipython-input-5-24552a212cef>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model_name in tqdm_notebook(model_names, total=len(model_names)):


  0%|          | 0/11 [00:00<?, ?it/s]

<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/170 [00:00<?, ?it/s]

C:\Users\ciara\anaconda3\envs\thesis\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


...

  0%|          | 0/170 [00:00<?, ?it/s]

D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\roc_orig_loss_1.png already exists, not saving image
D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\prc_orig_loss_1.png already exists, not saving image


<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/170 [00:00<?, ?it/s]

D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\roc_orig_loss_2.png already exists, not saving image
D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\prc_orig_loss_2.png already exists, not saving image


In [9]:
base_dir = 'D:/ucl_masters_data/project/augmentation_experiments'
run_and_plot_metrics(base_dir, test_set='alt',best_run='loss', model_names=None)

<ipython-input-5-24552a212cef>:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for model_name in tqdm_notebook(model_names, total=len(model_names)):


  0%|          | 0/11 [00:00<?, ?it/s]

<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/98 [00:00<?, ?it/s]

saved
saved


...

  0%|          | 0/98 [00:00<?, ?it/s]

D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\roc_alt_loss_1.png already exists, not saving image
D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\prc_alt_loss_1.png already exists, not saving image


<ipython-input-2-05eafe167ef3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, data in tqdm_notebook(enumerate(dataloader), total = len(dataloader)):


  0%|          | 0/98 [00:00<?, ?it/s]

D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\roc_alt_loss_2.png already exists, not saving image
D:/ucl_masters_data/project/augmentation_experiments\U_stdaug_A_none_L_simple\results\auc_curves\prc_alt_loss_2.png already exists, not saving image
